In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import BernoulliNB

In [2]:
train = pd.read_csv("seasons-1314-1718.csv")

predict = pd.read_csv("season-1819_csv.csv") 

train_copy = train.copy()

train_copy = train_copy.drop(['Div','Date','FTAG','FTR','HTHG','HTAG','HTR','HS','AS','HST','AST','HF','AF','HC','AC','HY','AY','HR','AR'],axis = 1)

train_copy.head()

,HomeTeam,AwayTeam,FTHG,Referee,B365H,B365D,B365A,BWH,BWD,BWA,...,BbAv<2.5,BbAH,BbAHh,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA,PSCH,PSCD,PSCA
0,Arsenal,Aston Villa,1,A Taylor,1.44,4.75,8.0,1.36,5.00,7.75,...,2.37,NaN,NaN,NaN,NaN,NaN,NaN,1.44,5.00,8.05
1,Liverpool,Stoke,1,M Atkinson,1.40,5.00,9.5,1.40,4.33,8.25,...,2.02,NaN,NaN,NaN,NaN,NaN,NaN,1.42,4.62,10.19
2,Norwich,Everton,2,M Oliver,3.20,3.40,2.4,3.10,3.25,2.30,...,1.82,NaN,NaN,NaN,NaN,NaN,NaN,3.81,3.27,2.21
3,Sunderland,Fulham,0,N Swarbrick,2.30,3.40,3.4,2.25,3.20,3.25,...,1.77,NaN,NaN,NaN,NaN,NaN,NaN,2.52,3.23,3.16
4,Swansea,Man United,1,P Dowd,4.20,3.50,2.0,4.10,3.50,1.87,...,1.98,NaN,NaN,NaN,NaN,NaN,NaN,3.62,3.41,2.22


In [476]:
train_labels = train['FTHG']

train_copy = train_copy.dropna(axis='columns')

data_dum = pd.get_dummies(train_copy, columns = train_copy.select_dtypes(include=['object']).columns.difference(['FTHG']))

data_dum = data_dum.drop('FTHG',axis=1)

X_train, X_test, y_train, y_test = train_test_split(data_dum,train_labels,test_size = 0.3, random_state = 0)

model = KNeighborsClassifier(n_neighbors = 15)
model = DecisionTreeClassifier(criterion="gini", max_depth=5, min_samples_leaf=500)
model = BernoulliNB(alpha=11)
model = RandomForestClassifier(n_estimators=150, criterion="gini", max_depth=5, random_state=0)

model.fit(X_train, y_train)
acc = accuracy_score(y_test, model.predict(X_test))

print(acc)

0.3684210526315789
